In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xs
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
import util 
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

In [ ]:
col_dict = {}
col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")

import pprint
uni_dict = col.unique(['source_id','experiment_id','table_id'])

In [ ]:
# Choose how much to coarsen data
coarsen_size = 2

# Define the common target grid axes
dlon, dlat = 1., 1.
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90.+dlat/2., 90., dlat)),
                     'lon': (['lon'], np.arange(0.+dlon/2., 360., dlon)),})

# Regridding function
def regrid_to_common(ds, ds_out):
    """
    Regrid from rectilinear grid to common grid
    """
    regridder = xe.Regridder(ds, ds_out, 'bilinear',periodic=True, reuse_weights=True)
    return regridder(ds)

Rearth = 6.378E6   # radius of Earth in meters
# a DataArray that gives grid cell areas on the lat/lon grid (in units of m^2)
area = (np.deg2rad(dlat)*Rearth) * (np.deg2rad(dlon)*Rearth*np.cos(np.deg2rad(ds_out.lat))) * xr.ones_like(ds_out.lon)

# coarsen
area = area.coarsen({'lat': coarsen_size, 'lon': coarsen_size}, boundary='exact').mean()

# month lengths
days_in_month = np.array([31 , 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
days_in_month = xr.DataArray(days_in_month, coords=[np.arange(1,13,1)], dims=['month'])

In [ ]:
models = set(uni_dict['source_id']['values']) # all the models
for experiment_id in ['historical', 'ssp585','ssp370']:
    query = dict(experiment_id=experiment_id, table_id='Amon',
                 variable_id='va', grid_label='gn')  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})
models = list(models)
models
cat = col.search(experiment_id=['historical'], table_id='Amon',
                 variable_id='va', grid_label='gn', source_id=models)   
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False})

In [ ]:
time_slice = slice('1950','2014')
plev_inter = 85000
region = dict(lat=slice(15,55),lon=slice(235,285))

In [ ]:
ds_dict = {}
for name, ds in tqdm(dset_dict.items()):
    if ('latitude' in ds.dims) and ('longitude'in ds.dims):
        ds = ds.rename({'latitude':'lat', 'longitude':'lon'}) #Some Models Labeled Differently
    ds = ds.mean(dim='member_id')
    ds = ds.sel(plev=plev_inter)
    ds = xr.decode_cf(ds)
    ds = ds.sel(time=time_slice)
    
    #regrid to a common grid    
    ds_new = regrid_to_common(ds['va'],ds_out) 
    
    # Create anomalies
    va_clim = ds_new.groupby('time.month').mean('time')
    va_anom = ds_new.groupby('time.month') - va_clim
    va_anom = va_anom - va_anom.sel(time=slice('1970','2000')).mean(dim='time')
    
    ## Calculate regional mean meridional wind
    va_anom_reg = va_anom.sel(**region)    
    va_anom_reg['name'] = ds.attrs['source_id']
            
    # Add ensemble as a new dimension
    #ds_Upd = va_anom_reg.expand_dims({'ensemble': np.array([va_anom_reg.attrs['name']])}, 0)
    
    # Add anomalous meridional wind to dictionary
    ds_dict[name] = va_anom_reg

In [ ]:
seasVa = {}
for va, ds in tqdm(ds_dict.items()):
    ds = ds.groupby('time.month').mean()
    ds = ds.sel(month=[4,5,6]).mean(dim='month').squeeze()
    
    seasVa[va] = ds

In [ ]:
big_plots = True
if big_plots:
    for name, ds in tqdm(seasVa.items()):
        fig = plt.figure(figsize=(9,7))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.add_feature(cartopy.feature.COASTLINE)
        ax.add_feature(cartopy.feature.BORDERS, linestyle='-')
        ax.add_feature(cartopy.feature.STATES, edgecolor = 'black')
        q = (ds['va']).plot(x='lon',y='lat')
        plt.title(name+".va.850 ", y = 1.025)
        plt.savefig('../figures/va_'+name+'-850.png',dpi=100,bbox_inches='tight')